In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# create dataframes
important_features = pd.read_csv("out/important_features.csv")

# rename columns 
important_features = important_features.rename(columns={'feature': 'importance'})
important_features = important_features.rename(columns={'Unnamed: 0': 'feature'})

# add label column
important_features['feature_type'] = np.where(important_features['feature'].str.contains('thickness'), 1, 
                                       np.where(important_features['feature'].str.contains('area'), 2, None))


# generate histogram
with plt.xkcd():
    sns.catplot(data=important_features, kind="bar", x="importance", y="feature", height=20, hue="feature_type")

# to do:
# 1. make dataframe based on condition (if contains str(thickess) assign label 1, if (area) assign label 2, use as hue)

ValueError: min() arg is an empty sequence